In [102]:
from random import choice

edge_traversals_dict_1 = {
      ('A','B') : 6,
      ('A','D') : 6,
      ('B','C') : 6,
      ('C','D') : 6,
}

# would use make_nodes_dict(G)

nodes_dict_1 = {
    
    'A': { ('A','B'), ('A','D') },
    'B': { ('A','B'), ('B','C') },
    'C': { ('B','C'), ('C','D') },
    'D': { ('A','D'), ('C','D') }
}
   
    
edge_traversals_dict_2 = {
    ('A','B') : 2,
    ('A','D') : 1,
    ('B','C') : 1,
    ('C','D') : 2,
    ('E','A') : 1,
    ('E','B') : 1,
    ('E','C') : 1,
    ('E','D') : 1  
}

# would use make_nodes_dict(G)
   
    
nodes_dict_2 = {
    'A': { ('A','B'), ('A','D'), ('A', 'E') },
    'B': { ('A','B'), ('B','C'), ('B', 'E') },
    'C': { ('B','C'), ('C','D'), ('C', 'E') },
    'D': { ('A','D'), ('C','D'), ('D', 'E') },
    'E': { ('A', 'E'),('B', 'E'), ('E','C'), ('E','D')}
}

print(nodes_dict_2['E'])

{('A', 'E'), ('E', 'D'), ('B', 'E'), ('E', 'C')}


In [103]:

# IDEA 1: make it so you don't go back to where you came from unless you have to 
    
# make a list of all edges - twice traverse gets in list twice 

def get_list_of_edges_by_traversal_count(edge_traversals_dict):
    """
    Given edge traversals dict, return a list of edges. 
    If the edge needs twice traversal it will appear twice in the 
    output list.
    
    
    NOTE: do i have to do something where you always choose returning to start node as last resort?
    """
    
    edges_list = []
    
    for edge in edge_traversals_dict:
        
        traversals = edge_traversals_dict[edge]
        
        for i in range(traversals):
        
            edges_list.append(edge)
        
#         if edge_traversals_dict[edge] == 2:
#             edges_list.append(edge)

    return edges_list


def get_route_order(start_node_, edge_traversals_dict_, nodes_dict_):
    
    node_order = []
    edges_traversed = []

    edges_list = get_list_of_edges_by_traversal_count(edge_traversals_dict_)
    
    print(edges_list)

    start_node = start_node_
    print(len(nodes_dict_[start_node_]))
    curr_node = start_node_ 
    
    previous_node = None
    
    while len(edges_list) > 0: # arbitrary while to start with

        print("\n\n----------------\nNOW AT:", curr_node)
        
        if len(node_order) > 0:
            previous_node = node_order[-1]
            
        print("previous_node:", previous_node)
        node_order.append(curr_node)
        print("nodes_order:", node_order)
        print("edges_list:", edges_list)

        print()
        # figure out where to go next
        # get all possible next nodes by looking at the connected edges

        possible_next_edges_all = list(nodes_dict_[curr_node]) # make set into list to support indexing
        
        print("possible_next_edges_all:", possible_next_edges_all)
        
        # make a list of possible next edges that don't include the previous node (ie. don't require 
        # doubling back)
        # make list of possible next edges that do include the previous node
        # add these lists together so that the edges that inlude the previous node are at the end of the list
        # because the getting of the next edge happens by looking at the start of the list first
        # SO doubling back is a last resort
        
        possible_next_inc_previous = []
        possible_next_no_inc_prev = []
        
        for edge in possible_next_edges_all:
            if previous_node in edge:
                possible_next_inc_previous.append(edge)
            elif previous_node not in edge:
                possible_next_no_inc_prev.append(edge)
                
                
        possible_next_edges = possible_next_no_inc_prev + possible_next_inc_previous
        
        # print("all possible next edges:", possible_next_edges)
        # print(f"possible next edges that don't inclue {previous_node}:", possible_next_edges_not_previous)
        
        # order possible next edges list so that doubling back to where you came from, 
        # ie. going back to the previous node is the last option

        keep_looking = True
        i = 0
        
        while keep_looking == True:
            
            print("\npossible_next_edges", possible_next_edges)

            print("current index, i:", i)

            # look through possible next edges for the current node 
            # if the edge is able to be traversed (still in edges_list)
            # make that edge the next edge_to_traverse and break out of loop
            # otherwise, try the next edge

            if possible_next_edges[i] in edges_list:

                edge_to_traverse = possible_next_edges[i]

                # remove the first instance of edge in list
                # (to account for edges that appear twice in list)
                edges_list.remove(edge_to_traverse)

                keep_looking = False


            # check if the reversed ordered edge is in the list 
            elif possible_next_edges[i][::-1] in edges_list:

                edge_to_traverse = possible_next_edges[i][::-1]

                # remove the first instance of edge in list
                # (to account for edges that appear twice in list)
                edges_list.remove(edge_to_traverse)

                keep_looking = False

            else: 
                if len(edges_list) > 0:
                    i = i + 1
                else:
                    keep_looking = False


        # make edge to traverse into list, so you can 
        # remove the current node and get a one-item list
        # then get that one item -- your next node

        print("next edge chosen:", edge_to_traverse)
        
        edge_to_traverse_lst = list(edge_to_traverse) # make edge to traverse tuple into a list
        
        edge_to_traverse_lst.remove(curr_node) # remove current node from edge to traverse list

        next_node = (edge_to_traverse_lst)[0] # next node is the only node left in the edge to traverse list

        edges_traversed.append(edge_to_traverse)
        print("edges_traversed:", edges_traversed)
        

        # make the next node your new curr node, and start the loop over
        curr_node = next_node
        print("next_node:", next_node)
            
            
# get_route_order('A', edge_traversals_dict_1, nodes_dict_1)
get_route_order('A', edge_traversals_dict_2, nodes_dict_2)

[('A', 'B'), ('A', 'B'), ('A', 'D'), ('B', 'C'), ('C', 'D'), ('C', 'D'), ('E', 'A'), ('E', 'B'), ('E', 'C'), ('E', 'D')]
3


----------------
NOW AT: A
previous_node: None
nodes_order: ['A']
edges_list: [('A', 'B'), ('A', 'B'), ('A', 'D'), ('B', 'C'), ('C', 'D'), ('C', 'D'), ('E', 'A'), ('E', 'B'), ('E', 'C'), ('E', 'D')]

possible_next_edges_all: [('A', 'D'), ('A', 'E'), ('A', 'B')]

possible_next_edges [('A', 'D'), ('A', 'E'), ('A', 'B')]
current index, i: 0
next edge chosen: ('A', 'D')
edges_traversed: [('A', 'D')]
next_node: D


----------------
NOW AT: D
previous_node: A
nodes_order: ['A', 'D']
edges_list: [('A', 'B'), ('A', 'B'), ('B', 'C'), ('C', 'D'), ('C', 'D'), ('E', 'A'), ('E', 'B'), ('E', 'C'), ('E', 'D')]

possible_next_edges_all: [('A', 'D'), ('C', 'D'), ('D', 'E')]

possible_next_edges [('C', 'D'), ('D', 'E'), ('A', 'D')]
current index, i: 0
next edge chosen: ('C', 'D')
edges_traversed: [('A', 'D'), ('C', 'D')]
next_node: C


----------------
NOW AT: C
previous_node: D
n

IndexError: list index out of range

In [29]:
# # get current node 

# curr_node = start_node

# node_order.append(curr_node)

# # get random edge from set of edges containing current node
# # make the set of edge tuples a list so you can use random.choice
# # make edge traversed a list so you can remove the curr edge in
# # order to get the next node

# edge_traversed = list(choice(list(nodes_dict[curr_node])))

# print(edge_traversed)

# edges_traversed.append(edge_traversed)

# edge_traversed.remove(curr_node)

# next_node = edge_traversed[0]

# print(next_node)

# # now we are at B OR D
# curr_node = next_node 
# node_order.append(curr_node)
# edge_traversed = list(choice(list(nodes_dict[curr_node])))
# next_node = edge_traversed.remove(curr_node)

## ------ scraps
    
    # get random edge from set of edges containing current node
    # NOTE: make the set of edge tuples a list so you can use random.choice
        
    # while the edge is not in the edges_list (ie. it can't be traversed)
    # keep choosing another edge
        
    # edge_traversed = choice(list(nodes_dict[curr_node]))
    # add edge traversed to edges_traversed list
    
    # make edge traversed a list so you can remove the curr edge in
    # order to get the next node
    edge_traversed_lst = list(edge_traversed)

    print(edge_traversed_lst)

    # get next node by removing currnt node from edge_traversed list 
    # (tuple that was turned into a list)
